#### Planteamiento del Problema

Se desea predecir la cantidad de puntos que pueda hacer un jugador en un partido, 
a partir de estadísticas personales (del jugador) y colectivas (del equipo en donde juega). 

Imports

In [1]:
#Se importan todas las dependencias necesarias
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2
import csv
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from nba_api.stats.endpoints import playergamelogs, teamgamelogs, teamgamelog, playerdashboardbylastngames

Cargo el DataFrame con los datos de data_raw.csv 

In [ ]:
players = pd.read_csv(r'C:\Users\sebas\Documents\Proyecto_DS\ProyectoDS\data\raw\data_raw.csv')
players.head()


Removemos las columnas que no se utilizarán en la generación del modelo predictivo.

In [ ]:
players.columns

In [ ]:
#Remocion de columnas
players = players.drop(['WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK','NBA_FANTASY_PTS', 'DD2',
       'TD3'], axis = 1)

players.head(2)

Tomamos el promedio de los últimos 5 partidos para las estadísticas por jugador

In [ ]:
#Ordenar el Dataframe de forma ascendente en base a las columnas descritas.
players = players.sort_values(by=['SEASON_YEAR','PLAYER_ID','GAME_DATE'], ascending=[True,True,True])

In [ ]:
#Hacemos reset el indice del dataframe en caso de que lo podamos necesitar en orden
players = players.reset_index(drop=True)

In [ ]:
players.shape
#contar la cantidad de datos distintos a NaN
sum(players.MIN_PROM.isna())

In [ ]:
def calcular_promedio_puntos(dataframe, columna, player_id, game_date, n_partidos):
    """
    Calcula el promedio de puntos de un jugador en los N partidos anteriores, excluyendo el partido actual.
    Parámetros:
      dataframe: El dataframe con los datos de los jugadores.
      game_date: El GAME_ID del partido actual.
      player_id: El PLAYER_ID del jugador para el que se quiere calcular el promedio.
      n_partidos: El número de partidos anteriores a considerar.
      columna: La columna que contiene los puntos del jugador.
    Devuelve:
      El promedio de puntos del jugador en los N partidos anteriores.
    """
    # Filtrar el dataframe para los partidos anteriores al actual y del jugador específico
    partidos_anteriores_jugador = dataframe[(dataframe['GAME_DATE'] < game_date) & (dataframe['PLAYER_ID'] == player_id)]
    
    # Si no hay suficientes partidos anteriores para el jugador, tomar los partidos disponibles
    n_partidos_disponibles = min(len(partidos_anteriores_jugador), n_partidos)
    
    # Si hay al menos un partido anterior, calcular el promedio de puntos
    if n_partidos_disponibles > 0:
        promedio_puntos = partidos_anteriores_jugador.tail(n_partidos_disponibles)[columna].mean()
    else:
        # Si no hay partidos anteriores disponibles, tomar el valor del partido actual
        promedio_puntos = dataframe[(dataframe['GAME_DATE'] == game_date) & (dataframe['PLAYER_ID'] == player_id)][columna].iloc[0]
    
    return promedio_puntos

#Lista de columnas para las cuales se obtendrán las estadísticas promediadas
lista_columnas_para_promedio = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'PTSRTeam',
       'PTSTeam', 'W_PCTTeam']

#Tomamos una listas de los jugadores y de las fechas de los partidos.
jugadores_ID = players.PLAYER_ID.unique().tolist()
dates = players.GAME_DATE.unique().tolist()


n_partidos = 5 #numero de partidos para sacar el promedio

#Se llama a la funcion para obtener las estadisticas promediadas de todos los jugadores
for columna in lista_columnas_para_promedio:
  for jugador in jugadores_ID:
      for date in dates:
        
        indice = players.index[(players['GAME_DATE']==date) & (players['PLAYER_ID']==jugador)]

        if indice.empty:
                # Si no se encontró un índice válido, pasar a la siguiente fecha
          continue

        promedio = calcular_promedio_puntos(players, columna, jugador, date, n_partidos)

        if promedio is None:
                # Pasar a la siguiente fecha
          continue
              
        players.loc[indice,columna+'_'+'PROM'] = promedio

In [ ]:
players.head()

In [ ]:
players.to_csv(r'C:\Users\sebas\Documents\Proyecto_DS\ProyectoDS\data\raw\data_complete_raw.csv', index=False)


Cargo el nuevo CSV completo.

In [2]:
players = pd.read_csv(r'C:\Users\sebas\Documents\Proyecto_DS\ProyectoDS\data\raw\data_complete_raw.csv')
players.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,STL_PROM,BLK_PROM,BLKA_PROM,PF_PROM,PFD_PROM,PTS_PROM,PLUS_MINUS_PROM,PTSRTeam_PROM,PTSTeam_PROM,W_PCTTeam_PROM
0,2020-21,2544,LeBron James,LeBron,1610612747,LAL,Los Angeles Lakers,22000002,2020-12-22T00:00:00,LAL vs. LAC,...,0.000000,0.00,1.000000,2.00,6.000000,22.000000,2.000000,116.000000,109.000000,0.00000
1,2020-21,2544,LeBron James,LeBron,1610612747,LAL,Los Angeles Lakers,22000008,2020-12-25T00:00:00,LAL vs. DAL,...,0.000000,0.00,1.000000,2.00,6.000000,22.000000,2.000000,116.000000,109.000000,0.00000
2,2020-21,2544,LeBron James,LeBron,1610612747,LAL,Los Angeles Lakers,22000040,2020-12-27T00:00:00,LAL vs. MIN,...,0.500000,0.00,1.000000,1.00,5.000000,22.000000,9.000000,115.500000,123.500000,0.25000
3,2020-21,2544,LeBron James,LeBron,1610612747,LAL,Los Angeles Lakers,22000045,2020-12-28T00:00:00,LAL vs. POR,...,0.666667,0.00,0.666667,1.00,5.333333,20.666667,11.333333,107.333333,124.666667,0.38900
4,2020-21,2544,LeBron James,LeBron,1610612747,LAL,Los Angeles Lakers,22000060,2020-12-30T00:00:00,LAL @ SAS,...,1.000000,0.25,1.000000,1.25,4.750000,22.750000,8.500000,109.250000,120.250000,0.41675
